In [1]:
import os
from tqdm import tqdm
import shutil
from pathlib import Path
import nibabel as nib
# import matlab.engine

In [2]:
AD_COHORT_DIR = r'C:Users\keena\School\Georgia Tech\FA25 Classes\CS 8903\cVAE\AD_Cohort_MRI'
AD_COHORT_FLAT_DIR = r'C:\Users\keena\School\Georgia Tech\FA25 Classes\CS 8903\cVAE\ad_cohort_mri_flat'

AD_COHORT_DEFORM_DIR = r'D:\multibrain-mri-y'


In [3]:
def get_mri_paths(cohort_dir):    
    for patient_id in os.listdir(cohort_dir):
        patient_id_path = os.path.join(cohort_dir, patient_id)
        for img_preproc_type in os.listdir(patient_id_path):
            img_preproc_path = os.path.join(patient_id_path, img_preproc_type)
            for visit_date in os.listdir(img_preproc_path):
                visit_date_path = os.path.join(img_preproc_path, visit_date)
                for img_id in os.listdir(visit_date_path):
                    img_id_path = os.path.join(visit_date_path, img_id)
                    for img_name in os.listdir(img_id_path):
                        full_path = os.path.join(img_id_path, img_name)
                        yield full_path


def move_to_flat_dir(adni_cohort_dir, out_dir):
    '''Convert ADNI MRI data that is in nested folders into a single flat directory'''
    for mri_path in get_mri_paths(adni_cohort_dir):
        shutil.copy2(mri_path, out_dir)

In [4]:
def apply_all_deformations_mri(imgs_dir, deformations_dir, out_dir):
    '''
    Apply deformations to the given sMRI images. Writes the result to `out_dir`.
    `imgs_dir` should be "flat", that is, all images stored in one folder with no subfolders.  
    '''
    if not os.path.exists(imgs_dir):
        raise ValueError(f'Image folder does not exist: {imgs_dir}')
    if not os.path.exists(deformations_dir):
        raise ValueError(f'Deformation folder does not exist: {deformations_dir}')
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
        print(f'Created new folder "{out_dir}"')
    
    eng = matlab.engine.start_matlab()
    eng.cd(os.getcwd())

    deformations_names = os.listdir(deformations_dir)

    for img_name in tqdm(os.listdir(imgs_dir)):
        def_fields_names = [x for x in deformations_names if Path(img_name).stem in x and x.startswith('y_')]
        if len(def_fields_names) != 1:
            continue
        #     raise ValueError(f'Should only find one deformation field for "{img_name}. Found {len(def_fields)}."')
        img_path = os.path.join(imgs_dir, img_name)
        def_path = os.path.join(deformations_dir, def_fields_names[0])

        out = eng.apply_deformation(img_path, def_path, out_dir)


def apply_all_deformations_pet(pet_imgs_dir, deformations_dir, out_dir, cohort_info_file):
    '''
    Apply deformations obtained from multibrain fit to PET images. Writes the results to `out_dir`.
    `pet_imgs_dir` should be "flat", that is, all images stored in one folder with no subfolders.  

    `cohort_info_file` is a CSV that should match each PET image with its closest MRI image in time for each 
    given patient. There should be an `image_id_pet` column and `image_id_mri` column.
    '''
    if not os.path.exists(pet_imgs_dir):
        raise ValueError(f'Image folder does not exist: {pet_imgs_dir}')
    if not os.path.exists(deformations_dir):
        raise ValueError(f'Deformation folder does not exist: {deformations_dir}')
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
        print(f'Created new folder "{out_dir}"')

    cohort_df = pd.read_csv(cohort_info_file)
    ad_cohort_df = cohort_df[cohort_df['DIAGNOSIS'] == 3.0]
    pet_ids, mri_ids = ad_cohort_df['image_id_pet'], ad_cohort_df['image_id_mri']

    eng = matlab.engine.start_matlab()
    eng.cd(os.getcwd())

    deform_names = os.listdir(deformations_dir)
    pet_names = os.listdir(pet_imgs_dir)

    for i in tqdm(list(range(len(ad_cohort_df)))):
        pet_id, mri_id = pet_ids.iloc[i], mri_ids.iloc[i]
        pet_img_name = [img_name for img_name in pet_names if re.search(rf'^({pet_id})', img_name) is not None]
        if len(pet_img_name) != 1:
            continue

        def_name = [img_name for img_name in deform_names if re.search(rf'I({mri_id})', img_name) is not None]
        if len(def_name) != 1:
            # print(mri_id)
            # raise ValueError()
            continue

        pet_img_path = os.path.join(pet_imgs_dir, pet_img_name[0])
        def_path = os.path.join(deformations_dir, def_name[0])

        out = eng.apply_deformation(pet_img_path, def_path, out_dir)
        

In [5]:
# move_to_flat_dir(AD_COHORT_DIR, AD_COHORT_FLAT_DIR)
# apply_all_deformations_mri(AD_COHORT_FLAT_DIR, 
#                            AD_COHORT_DEFORM_DIR, 
#                            r'C:\Users\keena\School\Georgia Tech\FA25 Classes\CS 8903\cVAE\ad_cohort_mri_warped',
#                            )

In [7]:
for img_name in os.listdir(AD_COHORT_FLAT_DIR):
    print(img_name)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\keena\\School\\Georgia Tech\\FA25 Classes\\CS 8903\\cVAE\\ad_cohort_mri_flat'

<class 'nibabel.nifti1.Nifti1Header'> object, endian='>'
sizeof_hdr      : 348
data_type       : np.bytes_(b'')
db_name         : np.bytes_(b'')
extents         : 0
session_error   : 0
regular         : np.bytes_(b'r')
dim_info        : 0
dim             : [  3 160 160  96   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.   1.5  1.5  1.5  0.   0.   0.   0. ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : np.bytes_(b'PT')
aux_file        : np.bytes_(b'')
qform_code      : scanner
sform_code      : scanner
quatern_b       : 0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoffset_x       : 0.0
qoffset_y       : -238.5
q